#### Load dependencies

In [ ]:
import numpy as np
import pandas as pd
import math

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors, KDTree
from sklearn.cluster import dbscan
from sklearn.neighbors import kneighbors_graph

from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_mutual_info_score, adjusted_rand_score
from scipy.sparse.csgraph import connected_components

import networkx as nx
import csv
from collections import Counter
from scipy.stats import gaussian_kde

import os

In [ ]:
# For dc-distance
from scipy.linalg import expm
import itertools
import numba

#### Functions

##### Data processing

In [ ]:
def read_data_npy(file, withoutnoise=False):
  data = np.load(file)
  n, d = data.shape

  if withoutnoise:
    data_nonoise = data[data[:, -1] != -1]
  else:
    data_nonoise = data

  X = data_nonoise[:, :d-1]
  y = data_nonoise[:, d-1]
  n, d = X.shape

  return X, y, n, d

In [ ]:
def plot_data_2D(X, axis1=0, axis2=1, title='DENSIRED'):
  plt.figure(figsize=(8,5))
  plt.scatter(X[:,int(axis1)], X[:,int(axis2)])
  plt.title(title)
  plt.show()

In [ ]:
# "elbow" method for choosing eps
def find_eps(data, k=2, title='Data'):

    neigh = NearestNeighbors(n_neighbors=k)
    nbrs = neigh.fit(data)
    distances, indices = nbrs.kneighbors(data)
    distances = np.sort(distances, axis=0)
    distances = distances[:, 1]

    fig = plt.figure(figsize=(8, 5))
    plt.plot(distances)
    plt.ylabel("eps")
    plt.title(title)
    plt.grid()
    plt.show()
    fig.savefig(title)

##### Distances

In [ ]:
# https://github.com/PhilJahn/DENSIRED/blob/main/distance_metric.py
class Component:
    def __init__(self, nodes, comp_id):
        self.nodes = set(nodes)
        self.comp_id = comp_id

def merge_components(c_i, c_j):
    merged_list = c_i.nodes.union(c_j.nodes)
    return Component(merged_list, c_i.comp_id)


def get_reach_dists(D, min_points, num_points):
    # Get reachability for each point with respect to min_points parameter
    reach_dists = np.sort(D, axis=1)
    reach_dists = reach_dists[:, min_points - 1]

    # Make into an NxN matrix
    reach_dists_i, reach_dists_j = np.meshgrid(reach_dists, reach_dists)

    # Take max of reach_i, D_ij, reach_j
    D = np.stack([D, reach_dists_i, reach_dists_j], axis=-1)
    D = np.max(D, axis=-1)

    # Zero out the diagonal so that it's a distance metric
    diag_mask = np.ones([num_points, num_points]) - np.eye(num_points)
    D *= diag_mask
    return D

def get_dc_dist_matrix(points, min_points=5, **kwargs):
    """
    We define the distance from x_i to x_j as min(max(P(x_i, x_j))), where
        - P(x_i, x_j) is any path from x_i to x_j
        - max(P(x_i, x_j)) is the largest edge weight in the path
        - min(max(P(x_i, x_j))) is the smallest largest edge weight
    """
    @numba.njit(fastmath=True, parallel=True)
    def get_dist_matrix(points, D, dim, num_points):
        for i in numba.prange(num_points):
            x = points[i]
            for j in range(i+1, num_points):
                y = points[j]
                dist = 0
                for d in range(dim):
                    dist += (x[d] - y[d]) ** 2
                dist = np.sqrt(dist)
                D[i, j] = dist
                D[j, i] = dist

        return D

    num_points = int(points.shape[0])
    dim = int(points.shape[1])
    density_connections = np.zeros([num_points, num_points])
    D = np.zeros([num_points, num_points])
    D = get_dist_matrix(points, D, dim, num_points)
    if min_points > 1:
        if min_points > num_points:
            raise ValueError('Min points cannot exceed the size of the dataset')
        D = get_reach_dists(D, min_points, num_points)

    flat_D = np.reshape(D, [num_points * num_points])
    argsort_inds = np.argsort(flat_D)

    component_dict = {i: Component([i], i) for i in range(num_points)}
    neighbor_dists = [[] for i in range(num_points)]
    neighbor_inds = [[] for i in range(num_points)]
    max_comp_size = 1
    for index in argsort_inds:
        i = int(index / num_points)
        j = index % num_points
        if component_dict[i].comp_id != component_dict[j].comp_id:
            epsilon = D[i, j]
            for node_i in component_dict[i].nodes:
                for node_j in component_dict[j].nodes:
                    density_connections[node_i, node_j] = epsilon
                    density_connections[node_j, node_i] = epsilon

            merged_component = merge_components(component_dict[i], component_dict[j])
            for node in merged_component.nodes:
                component_dict[node] = merged_component
            size_of_component = len(component_dict[i].nodes)
            if size_of_component > max_comp_size:
                max_comp_size = size_of_component
        if max_comp_size == num_points:
            break

    return np.array(density_connections)

##### Create graph

In [ ]:
# Create the epsilon graph based on dc-distance
def dc_graph(X, k, eps):
    n, d = X.shape

    # Get the density connections matrix
    density_connections = get_dc_dist_matrix(X, k)

    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in range(n):
        for j in range(i+1, n):
            if density_connections[i, j] <= eps:
                G.add_edge(i, j)


    return G

In [ ]:
def plot_eps_graph(X, G, dbscan_labels):
    plt.figure(figsize=(10, 8))
    pos = {i: X[i, :2] for i in G.nodes}
    node_colors = [dbscan_labels[i] for i in G.nodes]

    #https://memgraph.github.io/networkx-guide/visualization/basics/
    nx.draw(G,
            pos=pos,
            node_color=node_colors, node_size=10, cmap=plt.get_cmap('tab20'),
            edge_color='grey', style='-', width=0.3)

    plt.title('Epsilon Neighborhood Graph')
    plt.grid(True)
    plt.show()

##### Parameters

In [ ]:
def def_minpts(sep=20, end=101):
  return list(range(2, sep)) + list(range(sep, end, 5))

In [ ]:
def def_eps(minPts):
  eps_values = []

  for p in minPts:
    matrix=get_dc_dist_matrix(X,p)
    unique_values = np.round(np.unique(matrix),2)
    eps_values=np.concatenate((eps_values, unique_values))

  eps_values = np.unique(eps_values)
  if 0 in eps_values:
    eps_values = eps_values[1:]

  return eps_values

In [ ]:
# Computes the number of connected components in the graph
def cnt_connected_components(G, minPts=1, all=False):

  ''' If all is True, the number of connected components includes noise.
      If all is False, the number of connected components only includes those components with a size greater than minPts.'''

  connected_components = nx.connected_components(G)

  if all==True:
    return len([c for c in connected_components])

  filtered_components = [c for c in connected_components if len(c) > minPts]

  return len(filtered_components), filtered_components

In [ ]:
# Computes the number of cycles in the graph
# It's possiable to count cycles with different length
def cnt_simple_cycles(G, length_bound=3):
    cycles = list(nx.simple_cycles(G, length_bound=length_bound))
    return len(cycles)

In [ ]:
# Computes the number of cycles in the graph using the matrix
# https://math.stackexchange.com/questions/4337736/number-of-k-cycles-from-an-adjacency-matrix-of-a-graph
def cnt_loops(G, cnt_length=3):
  A = nx.to_numpy_array(G, nodelist=sorted(G.nodes()))

  n = A.shape[0]
  k = cnt_length
  eigenvalues = np.linalg.eigvals(A)
  trace = np.sum(eigenvalues)

  A_k = np.linalg.matrix_power(A, k)
  trace = np.trace(A_k)
  cycle_cnt = trace // 6

  return cycle_cnt

In [ ]:
def components_labels(components, X, toplot=True):
  component_labels = np.zeros(len(X), dtype=int)
  for label, component in enumerate(components):
    for node in component:
      component_labels[node] = label+1

  if toplot:
    component_labels = [-1 if x == 0 else x for x in component_labels]
  else:
    counter = -1
    for i in range(len(component_labels)):
      if component_labels[i] == 0:
        component_labels[i] = counter
        counter = counter - 1

  return component_labels

In [ ]:
def cnt_nmi(labels1, labels2):

  if len(labels1) == len(labels2):
    nmi = normalized_mutual_info_score(labels2, labels1)
  else:
    nmi = np.nan

  return nmi

In [ ]:
def cnt_ami(labels1, labels2):
    if len(labels1) == len(labels2):
        ami = adjusted_mutual_info_score(labels1, labels2)
    else:
        ami = np.nan

    return ami

In [ ]:
def cnt_ari(labels1, labels2):
    if len(labels1) == len(labels2):
        ari = adjusted_rand_score(labels1, labels2)
    else:
        ari = np.nan

    return ari

In [ ]:
# Compute the number of cycles in each connected component in the graph
#   and devided by the number of edges in this connected component
def cnt_loops_norm_by_edges_in_subgraphs(G):
  cc, components = cnt_connected_components(G)
  if cc == 0:
    return 0, 0

  subgraphs = [G.subgraph(comp).copy() for comp in components]

  s = 0
  for i, subgraph in enumerate(subgraphs):
    num_edges = subgraph.number_of_edges()
    cycles = cnt_loops(subgraph)

    if num_edges == 0:
      s += 0
    else:
      s = s + cycles/num_edges

  return round(s), round(s/cc)

In [ ]:
# Compute the average degree of the nodes
def cnt_avg_degree(G):
  cc, components = cnt_connected_components(G)
  if cc == 0:
    return 0

  subgraphs = [G.subgraph(comp).copy() for comp in components]

  s = 0
  for i, subgraph in enumerate(subgraphs):
    num_nodes = subgraph.number_of_nodes()
    average_degree = sum(dict(subgraph.degree()).values()) / num_nodes
    s += average_degree

  return round(s/cc)

In [ ]:
# Compute the maximum degree of the graph
def cnt_max_degree(G):
  cc, components = cnt_connected_components(G)
  if cc == 0:
    return 0

  subgraphs = [G.subgraph(comp).copy() for comp in components]
  global_max_degree = 0
  for i, subgraph in enumerate(subgraphs):
    max_degree = max(dict(subgraph.degree()).values())
    if max_degree > global_max_degree:
      global_max_degree = max_degree

  return global_max_degree

In [ ]:
# Compute the MST diameter
def cnt_mst_diameter(G):
   cc, components = cnt_connected_components(G)
   if cc == 0:
    return 0, 0, 0

   subgraphs = [G.subgraph(comp).copy() for comp in components]

   mst_diameter = []
   for i, subgraph in enumerate(subgraphs):
    mst = nx.minimum_spanning_tree(subgraph)
    mst_diameter.append(nx.diameter(mst) if nx.is_connected(mst) else float('inf'))

   return sum(mst_diameter), round(sum(mst_diameter)/cc), max(mst_diameter)

In [ ]:
# Compute the graph diameter
def cnt_graph_diameter(G):
   cc, components = cnt_connected_components(G)
   if cc == 0:
    return 0, 0, 0

   subgraphs = [G.subgraph(comp).copy() for comp in components]

   graph_diameter = []
   for i, subgraph in enumerate(subgraphs):
    graph_diameter.append(nx.diameter(subgraph) if nx.is_connected(subgraph) else float('inf'))

   return sum(graph_diameter), round(sum(graph_diameter)/cc), max(graph_diameter)

In [ ]:
def average_eps(arr, k=10):
    result = []
    n = len(arr)
    tr = 0.01*k

    for i in range(0, n, k):
        group = arr[i:i+k]
        result.append(np.mean(group))

    return np.round(result,2)

##### Dataset processing

In [ ]:

def cnt_parameters_dcdist_graph(X, y, minPts, eps_values, output_file='dcdist_results.csv'):
  file_exists = os.path.isfile(output_file) and os.path.getsize(output_file) > 0
  with open(output_file, mode='a', newline='') as file:
    writer = csv.writer(file)

    file.seek(0)
    if not file_exists:
      writer.writerow(['minPts', 'eps', 'num_connected_components', 'num_loops', 'nmi', 'ami', 'ari', 'cnt_loops_norm_by_edges', 'cnt_loops_norm_by_edges_comp', 'graph_labels'])


    num_components=[]
    num_loops =[]

    nmi_values = []
    ami_values = []
    ari_values = []

    num_loops_norm_by_edges = []
    num_loops_norm_by_edges_comp = []
    graph_labels = []


    for p in minPts:
      row_cc = []
      row_loops=[]

      row_nmi = []
      row_ami = []
      row_ari = []

      row_num_loops_norm_by_edges = []
      row_num_loops_norm_by_edges_comp = []
      row_graph_labels = []

      for e in eps_values:
        epsilon_graph = dc_graph(X, p, e)

        cc, components = cnt_connected_components(epsilon_graph)
        row_cc.append(cc)

        loops = cnt_loops(epsilon_graph)
        row_loops.append(loops)

        cnt_loops_norm_by_edges, cnt_loops_norm_by_edges_comp = cnt_loops_norm_by_edges_in_subgraphs(epsilon_graph)
        row_num_loops_norm_by_edges.append(cnt_loops_norm_by_edges)
        row_num_loops_norm_by_edges_comp.append(cnt_loops_norm_by_edges_comp)

        component_labels = components_labels(components, X, toplot=False)
        row_graph_labels.append(component_labels)

        nmi = cnt_nmi(component_labels, y)
        row_nmi.append(nmi)

        ami = cnt_ami(component_labels, y)
        row_ami.append(ami)

        ari = cnt_ari(component_labels, y)
        row_ari.append(ari)

        writer.writerow([p, e, cc, loops, nmi, ami, ari,
                         cnt_loops_norm_by_edges, cnt_loops_norm_by_edges_comp, component_labels])
        file.flush()

        print(p,e, cc, loops, nmi, ami, ari,
              cnt_loops_norm_by_edges, cnt_loops_norm_by_edges_comp, len(component_labels))

      num_components.append(row_cc)
      num_loops.append(row_loops)

      nmi_values.append(row_nmi)
      ami_values.append(row_ami)
      ari_values.append(row_ari)

      num_loops_norm_by_edges.append(row_num_loops_norm_by_edges)
      num_loops_norm_by_edges_comp.append(row_num_loops_norm_by_edges_comp)
      graph_labels.append(row_graph_labels)

  return num_components, num_loops, nmi_values, ami_values, ari_values, num_loops_norm_by_edges, num_loops_norm_by_edges_comp, graph_labels

##### Analysis

In [ ]:
def plot_heatmaps_connectedcomponent(num_components, nmi_values, eps_values, minPts, percent=90, highlight=False, save=False, name = 'num_components_nmi.png'):
  nmi_array = np.array(nmi_values)
  threshold = np.percentile(nmi_array, percent)
  print("NMI Threshold:", threshold)

  fig = plt.figure(figsize=(35, 10))
  sns.heatmap(num_components, xticklabels=np.round(eps_values, 2), yticklabels=minPts, annot=False, fmt=".0f", cmap="viridis")

  if highlight:

    for i in range(len(minPts)):
      for j in range(len(eps_values)):
          if  nmi_values[i][j] >= threshold:
              plt.gca().add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='red', lw=2))

  plt.xlabel('Epsilon')
  plt.ylabel('Minimum Samples')
  plt.title('Number of Connected Components by Epsilon and Min Samples')
  plt.show()
  if save:
    fig.savefig(name)

In [ ]:
def plot_heatmaps_loops(num_components, loops, nmi_values, eps_values, minPts, percent=90, highlight=False, save=False, name='loops_nmi.png'):
  nmi_array = np.array(nmi_values)
  threshold = np.percentile(nmi_array, percent)
  print("NMI Threshold:", threshold)

  fig = plt.figure(figsize=(35, 10))
  sns.heatmap(loops, xticklabels=np.round(eps_values, 2), yticklabels=minPts, annot=False, fmt=".0f", cmap="viridis")

  if highlight:

    for i in range(len(minPts)):
      for j in range(len(eps_values)):
          if  nmi_values[i][j] >= threshold:
              plt.gca().add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='red', lw=2))

  plt.xlabel('Epsilon')
  plt.ylabel('Minimum Samples')
  plt.title('Number of Cycles by Epsilon and Min Samples')
  plt.show()

  if save:
    fig.savefig(name)

In [ ]:
def plot_heatmaps_connectedcomponent_true(num_components, ground_truth_clusters, eps_values, minPts, percent=90, highlight=False, save=False):

  fig = plt.figure(figsize=(35, 10))
  sns.heatmap(num_components, xticklabels=np.round(eps_values, 2), yticklabels=minPts, annot=False, fmt=".0f", cmap="viridis")

  if highlight:

    for i in range(len(minPts)):
      for j in range(len(eps_values)):
          if  num_components[i][j] in ground_truth_clusters:
            plt.gca().add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='red', lw=2))

  plt.xlabel('Epsilon')
  plt.ylabel('Minimum Samples')
  plt.title('Number of Connected Components by Epsilon and Min Samples Ground True')
  plt.show()

  if save:
    name = '/content/drive/MyDrive/Colab Notebooks/num_components_nmi.png'
    fig.savefig(name)


In [ ]:
def plot_heatmaps_loops_true(num_components, loops, ground_truth_clusters, eps_values, minPts, percent=90, highlight=False, save=False):

  fig = plt.figure(figsize=(18, 14))
  sns.heatmap(loops, xticklabels=np.round(eps_values, 2), yticklabels=minPts, annot=False, fmt=".0f", cmap="viridis")

  if highlight:

    for i in range(len(minPts)):
      for j in range(len(eps_values)):
          if  num_components[i][j] in ground_truth_clusters:
            plt.gca().add_patch(plt.Rectangle((j, i), 1, 1, fill=False, edgecolor='red', lw=2))

  plt.xlabel('Epsilon')
  plt.ylabel('Minimum Samples')
  plt.title('Number of Connected Components by Epsilon and Min Samples Ground True')
  plt.show()

  if save:
    name = '/content/drive/MyDrive/Colab Notebooks/num_components_nmi.png'
    fig.savefig(name)


In [ ]:
def read_parameters(input_file):
    minPts = []
    eps_values = []
    num_components = []
    num_loops = []

    nmi_values = []
    ami_values = []
    ari_values = []

    num_loops_norm_by_edges = []
    num_loops_norm_by_edges_comp = []
    graph_labels = []


    with open(input_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)
        current_minPts = None
        row_cc = []
        row_loops = []

        row_nmi = []
        row_ami = []
        row_ari = []

        row_num_loops_norm_by_edges = []
        row_num_loops_norm_by_edges_comp = []
        row_graph_labels = []

        for row in reader:
            try:
                p = int(float(row[0]))
                e = float(row[1])
                cc = int(float(row[2]))
                loops = int(float(row[3]))
                nmi = float(row[4])
                ami = float(row[5])
                ari = float(row[6])
                loops_norm_by_edges = int(float(row[7]))
                loops_norm_by_edges_comp = int(float(row[8]))
                gr_labels = eval(row[9])
                #p, e, cc, loops, nmi, ami, ari, loops_norm_by_edges, loops_norm_by_edges_comp, gr_labels = int(float(row[0])), float(row[1]), int(float(row[2])), int(float(row[3])), float(row[4]), int(float(row[5])), int(float(row[6])), int(float(row[7])), int(row[8]), row[9]

            except ValueError as ve:
                print(f"ValueError: {ve} with row {row}")
                continue

            if p != current_minPts:
                if current_minPts is not None:
                    minPts.append(current_minPts)
                    num_components.append(row_cc)
                    num_loops.append(row_loops)

                    nmi_values.append(row_nmi)
                    ami_values.append(row_ami)
                    ari_values.append(row_ari)

                    num_loops_norm_by_edges.append(loops_norm_by_edges)
                    num_loops_norm_by_edges_comp.append(loops_norm_by_edges_comp)
                    graph_labels.append(row_graph_labels)

                current_minPts = p
                row_cc = []
                row_loops = []
                row_nmi = []
                row_ami = []
                row_ari = []
                row_num_loops_norm_by_edges = []
                row_num_loops_norm_by_edges_comp = []
                row_graph_labels = []

            row_cc.append(cc)
            row_loops.append(loops)
            row_nmi.append(nmi)
            row_ami.append(ami)
            row_ari.append(ari)
            row_num_loops_norm_by_edges.append(loops_norm_by_edges)
            row_num_loops_norm_by_edges_comp.append(loops_norm_by_edges_comp)
            row_graph_labels.append(gr_labels)

            if e not in eps_values:
                eps_values.append(e)

        # Append the last set of rows
        if current_minPts is not None:
            minPts.append(current_minPts)
            num_components.append(row_cc)
            num_loops.append(row_loops)
            nmi_values.append(row_nmi)
            ami_values.append(row_ami)
            ari_values.append(row_ari)
            num_loops_norm_by_edges.append(row_num_loops_norm_by_edges)
            num_loops_norm_by_edges_comp.append(row_num_loops_norm_by_edges_comp)
            graph_labels.append(row_graph_labels)

    return minPts, eps_values, num_components, num_loops, nmi_values, ami_values, ari_values, num_loops_norm_by_edges, num_loops_norm_by_edges_comp, graph_labels

In [ ]:
def print_parameters_specific_minPts_eps_dcgraph(X, y, minPts, eps, graph_type='dcdist', with_subgraphs=True):
  '''graph_type one of [dcdist, mrdist, knn] '''
  if graph_type == 'dcdist':
    epsilon_graph = dc_graph(X, minPts, eps)
  if graph_type == 'mrdist':
    epsilon_graph = mutual_reachability_dist_graph(X, minPts, eps)
  if graph_type == 'knn':
    epsilon_graph = knn_graph(X, minPts, eps)

  cc, components = cnt_connected_components(epsilon_graph)
  print(f'Number of connected componenet: {cc}')

  loops = cnt_loops(epsilon_graph)
  print(f'Number of loops: {loops}')

  compon_labels = components_labels(components, X)
  nmi = cnt_nmi(compon_labels, y)
  print(f'NMI: {nmi}')

  ami = cnt_ami(compon_labels, y)
  print(f'AMI: {ami}')

  ari = cnt_ari(compon_labels, y)
  print(f'ARI: {ari}')

  num_edges = epsilon_graph.number_of_edges()
  print(f'Number of edges: {num_edges}')

  print(f'loops/edges: {loops/num_edges}')

  component_labels = components_labels(components, X, toplot=True)

  plot_eps_graph(X, epsilon_graph, component_labels)

  if with_subgraphs:
    subgraphs = [epsilon_graph.subgraph(comp).copy() for comp in components]
    s = 0
    for i, subgraph in enumerate(subgraphs):
        print(f"Component {i + 1}:")

        num_nodes = subgraph.number_of_nodes()
        num_edges = subgraph.number_of_edges()
        print(f"Vertices: {num_nodes} Edges: {num_edges}")

        cycles = cnt_loops(subgraph)
        print("Cycles", cycles)
        if cycles != 0:
          s += cycles / num_edges
        print("loops/edges:" , cycles / num_edges)

        average_degree = sum(dict(subgraph.degree()).values()) / num_nodes
        print(f"Average Degree: {average_degree}")
        max_degree = max(dict(subgraph.degree()).values())
        print(f"Max Degree: {max_degree}")

        mst = nx.minimum_spanning_tree(subgraph)
        mst_diameter = nx.diameter(mst) if nx.is_connected(mst) else float('inf')
        print(f"MST Diameter: {mst_diameter}")

        graph_diameter = nx.diameter(subgraph) if nx.is_connected(subgraph) else float('inf')
        print(f"Graph Diameter: {graph_diameter}")
        print("\n")

    print("sum(loops/edges):" , s)


In [ ]:
def make_slices(arr, minPts, eps, eps_start, eps_end, minPts_start, minPts_end):
  eps_indices = [i for i, val in enumerate(eps) if eps_start <= val <= eps_end]
  minPts_indices = [i for i, val in enumerate(minPts) if minPts_start <= val <= minPts_end]

  array_slice = [[row[j] for j in eps_indices] for i, row in enumerate(arr) if i in minPts_indices]
  return array_slice

In [ ]:
def frequency_plot(arr, lim_start=-1, lim_end=30, save=False):
  flattened_array = list(itertools.chain.from_iterable(arr))
  frequency_count = Counter(flattened_array)

  x_values = list(frequency_count.keys())
  y_frequencies = list(frequency_count.values())

  fig, ax = plt.subplots(figsize=(10, 6))
  ax.bar(x_values, y_frequencies, color='skyblue', alpha=1, label='Histogram')

  kde = gaussian_kde(flattened_array)
  x_grid = np.linspace(np.min(flattened_array), np.max(flattened_array), 1000)
  kde_values = kde(x_grid)

  # Plot the KDE curve
  #ax.plot(x_grid, kde_values * len(flattened_array), color='red', lw=1 )#label='KDE Fit')

  # mean and median
  mean_value = np.mean(flattened_array)
  median_value = np.median(flattened_array)
  ax.axvline(mean_value, color='green', linestyle='--', lw=0.5, label=f'Mean: {mean_value:.2f}')
  ax.axvline(median_value, color='purple', linestyle='--', lw=0.5, label=f'Median: {median_value:.2f}')

  ax.set_xlabel('Values')
  ax.set_ylabel('Frequencies')
  #ax.set_yticks([])
  ax.set_xlim(lim_start,lim_end)
  ax.set_title('Frequency of Each Connected Component Value')
  ax.legend()
  #ax.grid(True)

  plt.tight_layout()
  plt.show()

  if save:
    fig.savefig("Frequency_of_connected_component.png")

  return frequency_count.most_common()

In [ ]:
def arr_value_specific_component(arr, num_component, n_comp):
  return arr[num_component == n_comp]

In [ ]:
def nmi_sort(components, nmi):

  num_components_flat = np.array(components).flatten()
  nmi_flat = np.array(nmi).flatten()

  sorted_indices = np.argsort(nmi_flat)[::-1]
  sorted_components = num_components_flat[sorted_indices]
  unique_values, indices = np.unique(sorted_components, return_index=True)

  sorted_indices = np.sort(indices)

  unique_sorted_components = sorted_components[sorted_indices]

  return unique_sorted_components


In [ ]:
def plot_cycles_relative_frequency_for_component(components, loops, unique_components, min_cycles=0, max_cycles=0, save=False, title='Cycles'):
  components = np.array(components)
  loops = np.array(loops)

  if max_cycles == 0:
    max_cycles = loops.max()
  if min_cycles == 0:
    min_cycles = loops.min()

  num_bins = 30
  bins = np.linspace(min_cycles, max_cycles, num_bins + 1)
  num_unique_components = len(unique_components)

  rows = (num_unique_components // 5) + 1
  cols = 5
  fig = plt.figure(figsize=(20, rows * 4))

  for i, component in enumerate(unique_components):
    plt.subplot(rows, cols, i + 1)

    cycles = loops[components == component]
    counts, _ = np.histogram(cycles, bins=bins)

    relative_frequency = counts / len(cycles)

    plt.bar(bins[:-1], relative_frequency, width=np.diff(bins), align='edge', alpha=0.75)
    plt.title(f'num_components = {component}')
    plt.xlabel('cycles')
    plt.ylabel('Relative Frequency')
    plt.xlim(min_cycles, max_cycles)
    plt.ylim(0, 1)
    plt.grid(True)

  plt.tight_layout()
  plt.show()

  if save:
    fig.savefig(title)


In [ ]:
def plot_cycles_relative_frequency_for_cycles_value(components, loops, unique_components, min_cycles=0, max_cycles=0, save=False, title='Cycles_relative'):
  components = np.array(components)
  loops = np.array(loops)

  if max_cycles == 0:
    max_cycles = loops.max()
  if min_cycles == 0:
    min_cycles = loops.min()

  num_bins = 30
  bins = np.linspace(min_cycles, max_cycles, num_bins + 1)
  num_unique_components = len(unique_components)

  rows = (num_unique_components // 5) + 1
  cols = 5
  fig = plt.figure(figsize=(20, rows * 4))

  max_frequency = 0

  for i, component in enumerate(unique_components):
    plt.subplot(rows, cols, i + 1)
    cycles = loops[components == component]
    relative_frequency = []

    #относительная величина, сколько раз конкретно это число встречается вовем массиве. Т.е. его концентрация для этого
    #конкретного компонента относительно всего массива
    for c in cycles:
        freq_value = len(loops[loops == c])
        relative_frequency.append(freq_value)
        if freq_value > max_frequency:
            max_frequency = freq_value

    counts, _ = np.histogram(cycles, bins=bins)

    #plt.bar(bins[:-1], relative_frequency, width=np.diff(bins), align='edge', alpha=0.75)
    plt.bar(bins[:-1], counts, width=np.diff(bins), align='edge', alpha=0.75)
    plt.title(f'num_components = {component}')
    plt.xlabel('cycles')
    plt.ylabel('Relative Frequency')
    plt.xlim(min_cycles, max_cycles)
    plt.ylim(0, max_frequency)
    plt.grid(True)

  plt.tight_layout()
  plt.show()
  if save:
    fig.savefig(title)


In [ ]:
def compute_nmi_per_component(components_list, num_connected_components, graph_labels):
  results={}
  for c in components_list:
    array = graph_labels[num_connected_components == c]

    nmi_values = []
    ami_values = []
    ari_values = []

    if len(array) > 1 and c != 0:
      for i in range(len(array)):
        for j in range(i+1, len(array)):
          nmi = normalized_mutual_info_score(array[i], array[j])
          nmi_values.append(nmi)
          #print(i,j,nmi)

          ami = adjusted_mutual_info_score(array[i], array[j])
          ami_values.append(ami)

          ari = adjusted_rand_score(array[i], array[j])
          ari_values.append(ari)

      results[c] = {
                'NMI': np.mean(nmi_values),
                'AMI': np.mean(ami_values),
                'ARI': np.mean(ari_values)
            }
    print(f"Num components: {c}, NMI: {np.mean(nmi_values):.3f}, AMI: {np.mean(ami_values):.3f}, ARI: {np.mean(ari_values):.3f}")

  return results

In [ ]:
def get_top_components_by_metric(results, metric, top_n=3):
    # Сортируем компоненты по значению метрики и выбираем топ-N
    sorted_components = sorted(results.items(), key=lambda x: x[1][metric], reverse=True)

    # Выводим топ-N результатов
    top_components = sorted_components[:top_n]

    print(f"Top {top_n} components by {metric}:")
    for component, metrics in top_components:
        print(f"Component: {component}, {metric}: {metrics[metric]:.3f}")

    return top_components

In [ ]:
def get_minpts_eps_pairs(k, num_components, minPts, eps_values):
  result = []
  for i, minPt in enumerate(minPts):
    for j, eps in enumerate(eps_values):
      if num_components[i][j] == k:
        result.append((minPt, eps))
  return result

In [ ]:

def read_dcdist_results(input_file='dcdist_results.csv'):
    minPts = []
    eps_values = []
    num_connected_components = []
    num_loops = []
    nmi_values = []
    ami_values = []
    ari_values = []
    cnt_loops_norm_by_edges = []
    cnt_loops_norm_by_edges_comp = []
    graph_labels = []

    with open(input_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Skip the header

        for row in reader:
            # Extracting the values from each row, and assuming data types as required
            minPts.append(float(row[0]))  # Assuming `minPts` is numeric
            eps_values.append(float(row[1]))  # Assuming `eps` is numeric
            num_connected_components.append(float(row[2]))  # Integer for `num_connected_components`
            num_loops.append(float(row[3]))  # Integer for `num_loops`
            nmi_values.append(float(row[4]))  # Assuming `nmi` is a floating-point number
            ami_values.append(float(row[5]))  # Assuming `ami` is a floating-point number
            ari_values.append(float(row[6]))  # Assuming `ari` is a floating-point number
            cnt_loops_norm_by_edges.append(float(row[7]))  # Assuming this is a floating-point number
            cnt_loops_norm_by_edges_comp.append(float(row[8]))  # Assuming this is a floating-point number

            # Assuming graph_labels is a list represented as a string in the CSV
            graph_labels.append(eval(row[9]))  # Convert string representation back to list
            #print(minPts, eps_values)

    return minPts, eps_values, num_connected_components, num_loops, nmi_values, ami_values, ari_values, cnt_loops_norm_by_edges, cnt_loops_norm_by_edges_comp, graph_labels



# **NPA data**

In [ ]:
file_name = 'gendata_50_15_1000_300.npy'
X, y, n, d = read_data_npy(file_name)
truth = len(np.unique(y))

#### Define minPts and eps values

In [ ]:
minPts = def_minpts(sep=20, end=21)
#print(len(minPts))
#print(minPts)

In [ ]:
eps_values = def_eps(minPts)
#print(len(eps_values))
#print(eps_values)

In [ ]:
len(eps_values) * len(minPts)

#### DC dist

In [ ]:
num_components_dc, num_loops_dc, nmi_values_dc, ami_values_dc, ari_values_dc, num_loops_norm_by_edges_dc, num_loops_norm_by_edges_comp_dc, graph_labels_dc = cnt_parameters_dcdist_graph(X, y, minPts, eps_values, 'dc_dist_parameters_50_15_1000_300_full.csv')

In [ ]:
plot_heatmaps_connectedcomponent(num_components_dc, nmi_values_dc, eps_values, minPts, highlight=True, save=True, name=file_name+'_component.csv')

In [ ]:
plot_heatmaps_loops(num_components_dc, num_loops_dc, nmi_values_dc, eps_values, minPts, highlight=True, save=True, name=file_name+'_loops.csv')

In [ ]:
plot_heatmaps_loops(num_components_dc, num_loops_norm_by_edges_dc, nmi_values_dc, eps_values, minPts, highlight=True, save=True, name=file_name+'_normloops.csv')
#np.max(np.array(num_loops_norm_by_edges_dc))

In [ ]:
top_cycles_dc = frequency_plot(num_loops_norm_by_edges_dc, lim_start=-1, lim_end=400, save=True)
#print(top_cycles_dc)

In [ ]:
top_dc = frequency_plot(num_components_dc, lim_start=-1, lim_end=50, save=True)
print(top_dc)

In [ ]:
components_most_frequency_dc = [value for value, _ in top_dc]
components_most_frequency_dc = components_most_frequency_dc[:]
print(components_most_frequency_dc)

In [ ]:
unique_sorted_components_dc = nmi_sort(num_components_dc, nmi_values_dc)
print(unique_sorted_components_dc)

In [ ]:
plot_cycles_relative_frequency_for_component(num_components_dc, num_loops_dc, components_most_frequency_dc, min_cycles=0, max_cycles=0, save=True, title='Cycles_dc')

In [ ]:
plot_cycles_relative_frequency_for_component(num_components_dc, num_loops_norm_by_edges_dc, components_most_frequency_dc, min_cycles=0, max_cycles=0, save=True, title='Cycles_norm_dc_rel')

In [ ]:
num_connected_components = np.array(num_components_dc)

graph_labels = np.array(graph_labels_dc)

nmi_ami_ari_dict = compute_nmi_per_component(np.unique(num_connected_components), num_connected_components, graph_labels)
print(nmi_ami_ari_dict)

In [ ]:
top_nmi = get_top_components_by_metric(nmi_ami_ari_dict, 'NMI', top_n=5)
top_ami = get_top_components_by_metric(nmi_ami_ari_dict, 'AMI', top_n=5)
top_ari = get_top_components_by_metric(nmi_ami_ari_dict, 'ARI', top_n=5)

In [ ]:
target_num_components = truth

In [ ]:
target_num_components = 15
result_pairs = get_minpts_eps_pairs(target_num_components, num_components_dc, minPts, eps_values)for i in result_pairs:
print('minPts:', i[0], 'eps:', i[1])